-  Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

In [702]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re 
from pymongo import MongoClient
import warnings

warnings.filterwarnings('ignore')

In [703]:
search = input('Введите название профессии: ')


while True:
     
    try:
        number = int(input('Введите количество страниц c которых необходимо собрать информацию (все: -1): '))
    except ValueError:
            print('\n' +'Не допустимое значение число должно быть больше "0" либо меньше "- 1"')
            continue
           
    if number is 0:
        print('\n' +'Не допустимое значение число должно быть больше "0" либо меньше "- 1"')
        continue
        
    if number is not 0:
        break
     
        
print('идет сбор информации время зависит от количества данных...')


i = 0

vacancies = pd.DataFrame()

while i is not number:
     
    main_link = 'https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&items_on_page=100&no_magic=true&search_field=name&page='+ str(i)
    
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
          'Accept':'*/*'}
    params = {'st':'searchVacancy', 'text':search}

    response = requests.get(main_link,headers=header,params=params).text

    dom = bs(response,'lxml')

    vacancy_list = dom.find_all('div',{'class':'vacancy-serp-item__row vacancy-serp-item__row_header'})

    for vacancy in vacancy_list:
        vacancy_data = pd.DataFrame()
        
    # name 
        vacancy_name = pd.Series(vacancy.find('a',{'class':'bloko-link HH-LinkModifier'}).getText())
        vacancy_data['name'] = vacancy_name
    
        vacancy_salary= vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
    
    #currency
        if vacancy_salary is None :
            vacancy_data['currency'] = None
        else:
            currency = re.findall('руб|USD|EUR', str(vacancy_salary))
            vacancy_data['currency'] = str(currency)[2:-2]
        
    # salary_min   
        if vacancy_salary is None :
            vacancy_data['salary_min'] = '0'
        else:
            vacancy_salary_min = re.sub(r'\s', "", str(vacancy_salary)) # убираем пробелы
            vacancy_salary_min1 = re.findall('от(\d+)', vacancy_salary_min) or \
            re.findall('(\d+)-', vacancy_salary_min)
            vacancy_data['salary_min'] = str(vacancy_salary_min1)[2:-2] 
            
         
            
        
      # salary_max   
        if vacancy_salary is None :
            vacancy_data['salary_max'] = '0'
        else:
            vacancy_salary_max = re.sub(r'\s', "", str(vacancy_salary)) # убираем пробелы
            vacancy_salary_max1 = re.findall('до(\d+)', vacancy_salary_max) or \
            re.findall('\d-(\d+)\w', vacancy_salary_max)
            vacancy_data['salary_max'] = str(vacancy_salary_max1)[2:-2] 
            
        
        
        
       # link_to_the_job 
        vacancy_link =vacancy.find('a',{'class':'bloko-link HH-LinkModifier'})['href']                             
        vacancy_data['link'] = vacancy_link
        vacancy_data ['Site']= 'hh.ru'
    
    
        vacancies = vacancies.append(vacancy_data)
    
    i += 1
    
    # button 
    try:
        button = dom.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href'] 
    except TypeError:
        break
        
vacancies.index = np.arange(1,(len(vacancies) + 1))  #  переназначаем индексы
vacancies.loc[vacancies['salary_min'] < '1', 'salary_min'] = '0'
vacancies.loc[vacancies['salary_max'] < '1', 'salary_max'] = '0'
vacancies['salary_min'] = vacancies['salary_min'].apply(int)
vacancies['salary_max'] = vacancies['salary_max'].apply(int)
vacancies

Введите название профессии: ruby
Введите количество страниц c которых необходимо собрать информацию (все: -1): 2
идет сбор информации время зависит от количества данных...


,name,currency,salary_min,salary_max,link,Site
1,Ruby on Rails Developer,USD,2000,0,https://hh.ru/vacancy/32954657?query=ruby,hh.ru
2,Ruby-on-Rails Developer,None,0,0,https://hh.ru/vacancy/36141459?query=ruby,hh.ru
3,Senior Ruby-on-Rails Developer (игровой проект),None,0,0,https://hh.ru/vacancy/35061448?query=ruby,hh.ru
4,"Разработчик Ruby, Ruby on Rails",руб,120000,180000,https://hh.ru/vacancy/36898378?query=ruby,hh.ru
5,Бэкенд-разработчик (Ruby on Rails),руб,120000,0,https://hh.ru/vacancy/31712815?query=ruby,hh.ru
...,...,...,...,...,...,...
118,Backend-разработчик - Ruby On Rails (Junior/Mi...,None,0,0,https://hh.ru/vacancy/36843697?query=ruby,hh.ru
119,Разработчик Ruby,None,0,0,https://hh.ru/vacancy/36756465?query=ruby,hh.ru
120,Backend разработчик (Ruby on Rails),руб,150000,0,https://hh.ru/vacancy/36754702?query=ruby,hh.ru
121,Разработчик Ruby junior,None,0,0,https://hh.ru/vacancy/36788541?query=ruby,hh.ru


In [704]:
client = MongoClient('localhost', 27017)
db = client['vacancies_database']         # создаем базу данных
collection_hh = db.hh  # создаем коллекцию


### - Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [705]:

def adding_values(vacancies):
    dict_vacancies_df = vacancies.T.to_dict() # преобразуем в словарь 

    i = 0

    while i < len(dict_vacancies_df):
        
        i+=1
        j = dict_vacancies_df.pop(i)
        collection_hh.update_one({'link':j['link']},{'$set':j},upsert=True)
        
adding_values(vacancies)

In [706]:
for i in collection_hh.find({}, {'name':1, 'currency':1, 'salary_min':1,'salary_max':1,'_id':0}).limit(3):
    print(i)

{'currency': 'USD', 'name': 'Ruby on Rails Developer', 'salary_max': 0, 'salary_min': 2000}
{'currency': None, 'name': 'Ruby-on-Rails Developer', 'salary_max': 0, 'salary_min': 0}
{'currency': None, 'name': 'Senior Ruby-on-Rails Developer (игровой проект)', 'salary_max': 0, 'salary_min': 0}


In [707]:
db.command("dbstats") # статистика по базе данных

{'db': 'vacancies_database',
 'collections': 1,
 'views': 0,
 'objects': 61,
 'avgObjSize': 181.45901639344262,
 'dataSize': 11069.0,
 'storageSize': 4096.0,
 'numExtents': 0,
 'indexes': 1,
 'indexSize': 4096.0,
 'scaleFactor': 1.0,
 'fsUsedSize': 106542006272.0,
 'fsTotalSize': 161152684032.0,
 'ok': 1.0}

### Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы

In [708]:
salary = int(input('введите значение заработной платы (без пробелов): '))

def search_salary(collection_hh, salary):
    vacancies = collection_hh.find({'$or': [{'salary_min':{'$gt':salary}}, {'salary_max':{'$gt':salary}}]})
    for vacancy in vacancies:
        pprint(vacancy)
        
search_salary(collection_hh, salary)       

введите значение заработной платы (без пробелов): 30000
{'Site': 'hh.ru',
 '_id': ObjectId('5ec80eb581ed8cabadfc8782'),
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/36898378?query=ruby',
 'name': 'Разработчик Ruby, Ruby on Rails',
 'salary_max': 180000,
 'salary_min': 120000}
{'Site': 'hh.ru',
 '_id': ObjectId('5ec80eb581ed8cabadfc8784'),
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/31712815?query=ruby',
 'name': 'Бэкенд-разработчик (Ruby on Rails)',
 'salary_max': 0,
 'salary_min': 120000}
{'Site': 'hh.ru',
 '_id': ObjectId('5ec80eb581ed8cabadfc8786'),
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/36731274?query=ruby',
 'name': 'Backend-разработчик Ruby on Rails',
 'salary_max': 110000,
 'salary_min': 70000}
{'Site': 'hh.ru',
 '_id': ObjectId('5ec80eb581ed8cabadfc8788'),
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/36787449?query=ruby',
 'name': 'Ruby on Rails Developer',
 'salary_max': 0,
 'salary_min': 160000}
{'Site': 'hh.ru',
 '_id': ObjectId('5ec80eb581

for i in db.collection_names():
    db.drop_collection(i)
    
    
db.command("dbstats")